In [7]:
import h5py
from New_grass import weighted_grassmannian_clustering
from tqdm import tqdm
import csv
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
import pickle
import scipy 

<div style="text-align:center; font-size:24px;">Loading the eigenvects and clustering data using grassmann clustering</div>

In [8]:
# Only dealing with data from sleep_run's for each subject
# All data has sample size 429
cluster_numbers = [2, 3, 4, 5, 6, 8, 10, 12, 15]
number_of_subjects = 33 
run_number = [2, 3, 4, 5] # sleep-run-numbers
all_data_matrices = {} # lib to hold all the sleep_stage data for each subject
valid_data_lib = {} # holds information about each subject and for what sleep stage runs we have data // valid_data_lib['01] - subject 1

for subject in range(1, number_of_subjects + 1):
    subject = str(subject).zfill(2) # fill 01, 02 and so on

    all_data_matrices[subject] = {}
    valid_data_lib[subject] = []

    for number in run_number:
        data_matrix = []
        with open(f"sleep_data/sleep_scores/sub-{subject}-sleep-stage.tsv", "r", newline="", encoding="utf-8") as tsv_file: # needs to do this for every number othervise it does not work
            tsv_reader = csv.reader(tsv_file, delimiter="\t")
            next(tsv_reader) # skip first row to remove column names
            for row in tsv_reader:
                if subject != "01":
                    if row[0] == f'task-sleep_run-{number}': 
                        data_matrix.append(row)
                else:
                    if row[1] == f'task-sleep_run-{number}': # We do this to handle the case where the first collumn is not 'task-sleep_run-x' as is the case for subject 1
                        data_matrix.append(row)   
            if len(data_matrix)>0:        
                all_data_matrices[subject][number] = data_matrix
                valid_data_lib[subject].append(number)

for number in run_number: # removing first collumn in data for subject "01" as it contains an extra collumn compared to the rest of the data
    try: 
        all_data_matrices["01"][number] = [row[1:] for row in all_data_matrices["01"][number]]
    except:
        continue

for subject in all_data_matrices: # removing subjects with no data for sleep stages at all
    numbers_to_remove = []
    for number in valid_data_lib[subject]: # removing sleep stages that contain a constant state 
        if len(np.unique([s[0] for s in np.array(all_data_matrices[subject][number])[:, 2]])) == 1:
            numbers_to_remove.append(number)
    for number in numbers_to_remove:
        valid_data_lib[subject].remove(number)
    if valid_data_lib[subject] == []:
        valid_data_lib.pop(subject)

# removing some subjects manually if they seem to contain to much noise or are awake almost all the time - this will not be caught above
valid_data_lib.pop('11')
valid_data_lib.pop('23')
valid_data_lib.pop('27')
valid_data_lib.pop('20')
valid_data_lib.pop('03')
valid_data_lib.pop('07')

valid_data_lib = {key: value for key, value in valid_data_lib.items() if len(value) >= len(run_number)} # finally we remove all subjects where we dont have valid data for all (4) runs

print(len(valid_data_lib)) # number of subjects left

13


In [9]:
# colleting all data an using vstack to get correct shape
all_data = []

for subject in valid_data_lib:
    for number in valid_data_lib[subject]:
        with h5py.File(f"sleep_data/eigvecs/sub-{subject}_session-task-sleep_run-{number}_eigvecs.h5", "r") as file:
            data = file['eigvecs'][:]
            all_data.append(data)

# Stack all the data vertically
all_data = np.vstack(all_data)

print(all_data.shape, 13*429*4)



(22308, 116, 2) 22308


In [10]:
#eigenvals
eigenvals = []
for subject in valid_data_lib:
    for number in valid_data_lib[subject]:
        with h5py.File(f"sleep_data/eigvals/sub-{subject}_session-task-sleep_run-{number}_eigvals.h5", "r") as file:
            data = file['eigvals'][:]
            eigenvals.append(data)
eigenvals = np.vstack(eigenvals)

print(eigenvals.shape, 13*429*4)

(22308, 2) 22308


In [11]:
def weighted_grassmannian_clustering(X,X_weights,K,number_of_subjects,max_iter=10000,tol=1e-16):
    """"
    Weighted grassmannian clustering using the chordal distance function and a SVD-based update rule
    
    X: size (nxpxq), where n is the number of observations, p is the number of features and q is the subspace dimensionality
    X_weights: size (n,q), where n is the number of observations and q is the subspace dimensionality (corresponds to eigenvalues)
    K: number of clusters
    max_iter: maximum number of iterations
    tol: tolerance for convergence

    """
    
    n,p,q = X.shape

    # initialize cluster centers using a normal distribution projected to the Grassmannian
    C = np.random.randn(K,p,q)
    C_weights = np.ones((K,q))
    for k in range(K):
        C[k] = C[k]@scipy.linalg.sqrtm(np.linalg.inv(C[k].T@C[k])) # project onto the Grassmannian

    # initialize counters
    data_number = 429
    sub_obj = []
    iter = 0
    obj = []
    partsum = np.zeros((max_iter,K))
    while True:
        # "E-step" - compute the similarity between each matrix and each cluster center
        dis = 1/np.sqrt(2)*(np.sum(X_weights**4)+np.sum(C_weights**4)-2*np.linalg.norm(np.swapaxes((X*X_weights[:,None,:])[:,None],-2,-1)@(C*C_weights[:,None,:])[None],axis=(-2,-1)))
        sim = -dis
        maxsim = np.max(sim,axis=1) # find the maximum similarity - the sum of this value is the objective function
        X_part = np.argmax(sim,axis=1) # assign each point to the cluster with the highest similarity
        obj.append(np.sum(maxsim))
        part_list = []
        for idx in range(number_of_subjects):
            part_list.append(np.sum(maxsim[idx*data_number : (idx+1)*data_number]))
        sub_obj.append(part_list)

        # check for convergence
        for k in range(K):
            partsum[iter,k] = np.sum(X_part==k)
        if iter>0:
            if all((partsum[iter-1]-partsum[iter])==0) or iter==max_iter or abs(obj[-1]-obj[-2])<tol:
                break
        
        # "M-step" - update the cluster centers
        for k in range(K):
            idx_k = X_part==k
            V = np.reshape(np.swapaxes(X[idx_k]*X_weights[idx_k,None,:],0,1),(p,np.sum(idx_k)*q))
            U,S,_ = scipy.sparse.linalg.svds(V,q)
            C[k] = U[:,:q]
            C_weights[k] = S**2

        iter += 1
    
    return C,obj,X_part,sub_obj

In [12]:
cluster_assignments_objfunc_cent = {}

for cluster in tqdm(cluster_numbers):
    cluster_assignments_objfunc_cent[cluster] = {}
    C, obj, part, sub_obj = weighted_grassmannian_clustering(all_data, eigenvals, cluster, len(valid_data_lib), max_iter=500)
    cluster_assignments_objfunc_cent[cluster]['C'] = C
    cluster_assignments_objfunc_cent[cluster]['obj'] = obj[-1] # tager kun den bedste obj value efter grassmann terminater
    cluster_assignments_objfunc_cent[cluster]['part'] = part
    cluster_assignments_objfunc_cent[cluster]['obj_sub'] = sub_obj[-1]

  0%|          | 0/9 [00:00<?, ?it/s]


ValueError: `A` must not be empty.

In [17]:
# Save the dictionary to a file
with open('cluster_assignments_weighted_objfunc.pkl', 'wb') as f:
    pickle.dump(cluster_assignments_objfunc_cent, f)

<div style="text-align:center; font-size:24px;">Using Objective Function Value as meassure for preformance</div>

<div style="text-align:center; font-size:24px;">Using NMI as meassure for preformance</div>


<div style="text-align:center; font-size:24px;">1 out of K encoding of the true labels</div>

<div style="text-align:center; font-size:24px;">1 out of K encoding of predicted labels using grassmann clustering</div>

<div style="text-align:center; font-size:24px;">Calculating NMI</div>